In [ ]:
!pip install keras_preprocessing
import re
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
import matplotlib.pyplot as plt
from keras_preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.optimizers import Adam, rmsprop
from sklearn.model_selection import  train_test_split
from tensorflow.keras.utils import plot_model
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.layers import Input, Embedding, Conv1D, Dropout, MaxPool1D, Flatten, Dense, Bidirectional, GRU
from tensorflow.keras.models import Model
from tensorflow.keras.layers import concatenate

In [2]:
import gensim.downloader
print(list(gensim.downloader.info()['models'].keys()))
w2v = gensim.downloader.load('word2vec-google-news-300')

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']
[=================================================-] 100.0% 1662.6/1662.8MB downloaded


In [3]:
df1 = pd.read_csv('/content/spam.csv', encoding='latin1')
df1 = df1.drop(df1.columns[[2, 3, 4]], axis=1)
df1.rename(columns={"v1": "Category", "v2": "Message"}, inplace=True)
df1.head(5)

Category                                            Message
0      ham  Go until jurong point, crazy.. Available only ...
1      ham                      Ok lar... Joking wif u oni...
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...
3      ham  U dun say so early hor... U c already then say...
4      ham  Nah I don't think he goes to usf, he lives aro...

In [ ]:
df2 = pd.read_csv('/content/SMSSpamCollection.tsv', names=['Category', 'Message'], sep='\t')
df2.head(5)

In [ ]:
df3 = pd.read_csv('/content/spamhamdataset.csv')
df3.head(5)

In [6]:
df = pd.concat([df1, df2, df3])
df.reset_index(drop=True, inplace=True)

In [7]:
def remove_hyperlink(word):
    return  re.sub(r"http\S+", "", word)

def to_lower(word):
    result = word.lower()
    return result

def remove_number(word):
    result = re.sub(r'\d+', '', word)
    return result

def remove_punctuation(word):
    result = word.translate(str.maketrans(dict.fromkeys(string.punctuation)))
    return result

def replace_newline(word):
    return word.replace('\n','')

def remove_stopwords(sentence):
    stop_words = set(stopwords.words("english"))
    words = sentence.split()
    words = [word for word in words if word not in stop_words]
    return " ".join(words)

def lemmatize_text(sentence):
    lemmatizer = WordNetLemmatizer()
    words = sentence.split()
    words = [lemmatizer.lemmatize(word) for word in words]
    return " ".join(words)

def remove_special_characters(word):
    result = re.sub(r'[^\w\s]', '', word)
    return result

def clean_up_pipeline(sentence):
    cleaning_utils = [remove_hyperlink,
                      replace_newline,
                      to_lower,
                      remove_number,
                      remove_punctuation,remove_special_characters]
    for o in cleaning_utils:
        sentence = o(sentence)
    return sentence
df["Message"]= df["Message"].apply(clean_up_pipeline)


In [8]:
df['Category'] = df.Category.map({'ham':0, 'spam':1})


In [9]:
df_spam = df[df['Category']==1]
df_ham = df[df['Category']==0]
df_ham_downsampled = df_ham.sample(df_spam.shape[0],random_state=0)
df_ham_downsampled.shape
df_balanced = pd.concat([df_ham_downsampled, df_spam])
df_balanced.shape

(5984, 2)

In [10]:
x_raw = []
y_raw = []
from keras.preprocessing.text import Tokenizer

for index, row in df_balanced.iterrows():
    y_raw.append(row['Category'])
    x_raw.append(row['Message'])

In [11]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_raw)
vocab_size = len(tokenizer.word_index) + 1
len(tokenizer.word_index)
sequences = tokenizer.texts_to_sequences(x_raw)
max_length = max(len(seq) for seq in sequences)
print("Maximum sequence length:", max_length)

Maximum sequence length: 3800


In [12]:
# Pad the sequence to have the same size
trunc_type='post'
padding_type='post'
training_padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print('Into a padded sequence:', training_padded[4])

Into a padded sequence: [1111 9797 3780 ...    0    0    0]


In [14]:
from sklearn.model_selection import cross_val_score, train_test_split
y=df_balanced['Category'].values

#X_train, X_test, y_train, y_test = train_test_split(training_padded, y, test_size = 0.2, random_state= 0)
X_train, y_train = training_padded,y

In [15]:
import gensim

embeddings_index = dict()

for word in w2v.key_to_index.keys():
    embeddings_index[word] = w2v[word]

print('Loaded %s word vectors.' % len(embeddings_index))


Loaded 3000000 word vectors.


In [16]:
embedding_matrix = np.zeros((vocab_size, 300))
for word, index in tokenizer.word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector
print(embedding_matrix.shape)


(59495, 300)


In [17]:
def ensemble_CNN_BiGRU(filters = 100, kernel_size = 3, activation='relu',
                   input_dim = None, output_dim=300, max_length = None, emb_matrix = None):

    # Channel 1D CNN
    input1 = Input(shape=(max_length,))
    embeddding1 = Embedding(input_dim=input_dim,
                            output_dim=output_dim,
                            input_length=max_length,
                            input_shape=(max_length, ),
                            # Assign the embedding weight with word2vec embedding marix
                            weights = [emb_matrix],
                            # Set the weight to be not trainable (static)
                            trainable = False)(input1)
    conv1 = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu',
                   kernel_constraint= MaxNorm( max_value=3, axis=[0,1]))(embeddding1)
    pool1 = MaxPool1D(pool_size=2, strides=2)(conv1)
    flat1 = Flatten()(pool1)
    drop1 = Dropout(0.5)(flat1)
    dense1 = Dense(10, activation='relu')(drop1)
    drop1 = Dropout(0.5)(dense1)
    out1 = Dense(1, activation='sigmoid')(drop1)

    # Channel BiGRU
    input2 = Input(shape=(max_length,))
    embeddding2 = Embedding(input_dim=input_dim,
                            output_dim=output_dim,
                            input_length=max_length,
                            input_shape=(max_length, ),
                            # Assign the embedding weight with word2vec embedding marix
                            weights = [emb_matrix],
                            # Set the weight to be not trainable (static)
                            trainable = False,
                            mask_zero=True)(input2)
    gru2 = Bidirectional(GRU(64))(embeddding2)
    drop2 = Dropout(0.5)(gru2)
    out2 = Dense(1, activation='sigmoid')(drop2)

    # Merge
    merged = concatenate([out1, out2])

    # Interpretation
    outputs = Dense(1, activation='sigmoid')(merged)
    model = Model(inputs=[input1, input2], outputs=outputs)

    # Compile
    model.compile( loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [ ]:
# Parameter Initialization
activations = ['relu']
filters = 100

model = ensemble_CNN_BiGRU( input_dim=vocab_size, max_length=max_length, emb_matrix=embedding_matrix)
# Train the model
result=model.fit([X_train,X_train],y_train, batch_size=50, epochs=10, verbose=1)



In [19]:
import pickle
pickle.dump(model,open("spam.pkl","wb"))

In [21]:
import pickle
pickle.dump(tokenizer,open("token.pkl","wb"))